In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset, DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("../src/")
import importlib
import structures
importlib.reload(structures)
from structures import *
import losses
importlib.reload(losses)
from losses import *
import trainer
importlib.reload(trainer)
from trainer import *
#dtype = torch.float64
dtype = torch.float32
#dtype = torch.double
torch.set_default_dtype(dtype)
torch.autograd.set_detect_anomaly(True)
# Set the device to CUDA if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

#import deepspeed
import random
sys.path.append("/mnt/home/yjo10/ceph/myutils")
import plt_utils as pu

Using device: cuda


In [2]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces

class NBodyTimeStepEnv(gym.Env):
    def __init__(self):
        super(NBodyTimeStepEnv, self).__init__()
        
        self.observation_space = spaces.Box(
            low=np.array([0, 0, 0, 1e-6], dtype=np.float64),
            high=np.array([1e6, 1e6, 1e4, 1e-1], dtype=np.float64),
            dtype=np.float64
        )
        
        self.action_space = spaces.Box(
            low=np.array([np.log(1e-6)], dtype=np.float64),
            high=np.array([np.log(1e-2)], dtype=np.float64),
            shape=(1,), dtype=np.float64
        )
        
        self.state = None
        self.prev_dt = 1e-4

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.state = self.get_initial_state()
        self.prev_dt = 1e-4
        info = {}
        return self.state.astype(np.float32), info

    def step(self, action):
        log_dt = np.clip(action[0], self.action_space.low[0], self.action_space.high[0])
        dt = np.exp(log_dt)

        new_state, energy_error = self.step_nbody_sim(self.state, dt)
        reward = -energy_error + 0.01 * dt

        self.state = new_state
        terminated = False  # use logic if your episode ends normally
        truncated = False   # use logic if episode ends due to time limits
        info = {}

        return new_state.astype(np.float32), float(reward), terminated, truncated, info


    def get_initial_state(self):
        return np.array([0.0, 1.0, 0.0, self.prev_dt], dtype=np.float64)

    def step_nbody_sim(self, state, dt):
        new_state = state.copy().astype(np.float64)
        new_state[3] = dt
        energy_error = np.random.rand() * 1e-3  # placeholder
        return new_state, energy_error

In [ ]:
from stable_baselines3 import DDPG

env = NBodyTimeStepEnv()
model = DDPG('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=100000)
model.save("ddpg_nbody_timestep")

2025-07-07 13:47:36.123865: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-07 13:47:36.165751: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX512F AVX512_VNNI, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [3]:
from stable_baselines3 import DDPG

# Re-create (or wrap) your env exactly as during training
env = NBodyTimeStepEnv()

# Load the agent; pass env if you want to continue interacting with it
model = DDPG.load("ddpg_nbody_timestep", env=env)

2025-07-07 17:10:26.776062: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX512F, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: Th

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [4]:
from stable_baselines3.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(
    model,                # your loaded agent
    env,                  # the same (or a fresh) env
    n_eval_episodes=10,   # e.g. run 10 full episodes
    deterministic=True,   # use the mean action from the policy
    render=False          # or True to watch it
)

print(f"Mean reward over 10 episodes: {mean_reward:.2f} ± {std_reward:.2f}")

/mnt/home/yjo10/pyenv/torch/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


KeyboardInterrupt: 

In [7]:
from stable_baselines3 import SAC

env = NBodyTimeStepEnv()
model = SAC('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10)
model.save("sac_nbody_timestep")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [11]:
obs

(array([0.e+00, 1.e+00, 0.e+00, 1.e-04], dtype=float32), {})

In [18]:
n_eval_episodes = 20
all_rewards = []
all_lengths = []

for ep in range(n_eval_episodes):
    obs, _ = env.reset()
    ep_reward = 0.0
    ep_length = 0

    #terminated = False
    #truncated = False
    #while not terminated and not truncated:
    for _ in range(10):
        action, _ = model.predict(obs, deterministic=True)
        print(action)
        obs, reward, terminated, truncated, info = env.step(action)
        ep_reward += reward
        ep_length += 1

    all_rewards.append(ep_reward)
    all_lengths.append(ep_length)

# Compute summary stats
import numpy as np

print(f"Average reward: {np.mean(all_rewards):.2f} ± {np.std(all_rewards):.2f}")
print(f"Average episode length: {np.mean(all_lengths):.1f} steps")

[-9.23460936]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460936]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460936]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460936]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460936]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460936]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460936]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460991]
[-9.23460936]
[-9.23